In [ ]:
!pip install cassandra-driver

In [ ]:
#!pip install fiddler-client==2.1.0

In [14]:
!pip install langchain

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [10]:
!pip install typing-inspect==0.8.0 typing_extensions==4.5.0

  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.52.0 requires pyarrow<12.0.0,>=3.0.0, but you have pyarrow 13.0.0 which is incompatible.
chromadb 0.3.26 requires tqdm>=4.65.0, but you have tqdm 4.64.1 which is incompatible.
nemoguardrails 0.5.0 require

In [8]:
pip install pydantic -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 692.6 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 4.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.8 MB/s eta 0:00:0000:0100:01
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.14.3
    Uninstalling pydantic_core-2.14.3:
      Successfully uninstalled pydantic_core-2.14.3
  Attempt

In [9]:
pip install pydantic==1.10.11

  Using cached pydantic-1.10.11-cp38-cp38-macosx_10_9_x86_64.whl.metadata (148 kB)
Using cached pydantic-1.10.11-cp38-cp38-macosx_10_9_x86_64.whl (2.9 MB)
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.5.2
    Uninstalling pydantic-2.5.2:
      Successfully uninstalled pydantic-2.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.3.26 requires tqdm>=4.65.0, but you have tqdm 4.64.1 which is incompatible.
fiddler-client 2.2.0 requires pydantic==1.9.0, but you have pydantic 1.10.11

In [10]:
!pip install flask-sqlalchemy

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install unstructured

In [16]:
! pip install -q --progress-bar off \
    "git+https://github.com/hemidactylus/langchain@updated-full-preview--lab#egg=langchain&subdirectory=libs/langchain" \
    "cassio>=0.1.1" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2" 
exit()

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [1]:
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
import os
import pandas as pd
import fiddler as fdl

In [2]:
fdl.__version__

'2.2.0'

In [3]:
cassandra.__version__

'3.28.0'

In [4]:
# This secure connect bundle is autogenerated when you donwload your SCB, 
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'datastax_auth/secure-connect-fiddlerai.zip'
}

ASTRA_DB_APPLICATION_TOKEN = os.environ.get('ASTRA_DB_APPLICATION_TOKEN')
#print("TOKEN: " + ASTRA_DB_APPLICATION_TOKEN)

# This token json file is autogenerated when you donwload your token, 
# if yours is different update the file name below
#with open("datastax_auth/danny@fiddler.ai-token.json") as f:
#    secrets = json.load(f)

#CLIENT_ID = secrets["clientId"]
#CLIENT_SECRET = secrets["secret"]

auth_provider=PlainTextAuthProvider("token", ASTRA_DB_APPLICATION_TOKEN)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)

#auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
#cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [5]:
session.set_keyspace('fiddlerai')

In [6]:
# Set your secret(s) for LLM access:
llmProvider = 'OpenAI'  # 'GCP_VertexAI', 'Azure_OpenAI'

In [7]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader, DirectoryLoader, DataFrameLoader

In [8]:
from langchain.vectorstores.cassandra import Cassandra

In [9]:
import os
import openai

In [10]:
os.environ['OPENAI_API_TYPE'] = 'open_ai'
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
llm = OpenAI(temperature=0)
myEmbedding = OpenAIEmbeddings()
print('LLM+embeddings from OpenAI')

LLM+embeddings from OpenAI


In [11]:
table_name = 'fiddler_doc_snippets_' + llmProvider

index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Cassandra,
    embedding=myEmbedding,
    text_splitter=CharacterTextSplitter(
        chunk_size=5000,
        chunk_overlap=0,
    ),
    vectorstore_kwargs={
        'session': session,
        'keyspace': 'fiddlerai',
        'table_name': table_name,
    },
)

In [12]:
df = pd.read_csv('documentation_data/vector_index_feed_23.6.csv')
df

,text
0,"---\ntitle: ""fdl.FiddlerApi""\nslug: ""client-se..."
1,"slug: ""client-setup"" 58'\n\nclient = fdl.Fiddl..."
2,"---\ntitle: ""Customer Churn Prediction""\nslug:..."
3,"slug: ""customer-churn-prediction"" images"": [\n..."
4,"slug: ""customer-churn-prediction"" f7d1-Churn-i..."
...,...
377,"Currently, only the following fields in [fdl.M..."
378,AI has been in the limelight thanks to ‌recent...
379,LLM means large language model. A large langu...
380,"The term generative AI, or GenAI, also is clos..."


In [13]:
loader = DataFrameLoader(df, page_content_column="text")

In [14]:
len(loader.load())

382

In [15]:
index = index_creator.from_loaders([loader])
index

VectorStoreIndexWrapper(vectorstore=<langchain.vectorstores.cassandra.Cassandra object at 0x7fed43f4eee0>)

In [45]:
cqlSelect = f'SELECT * FROM fiddlerai.{table_name} LIMIT 30;'  # (Not a production-optimized query ...)
rows = session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    # depending on the cassIO version, the underlying Cassandra table can have different structure ...
    try:
        # you are using the new cassIO 0.1.0+ : congratulations :)
        print(f'    row_id:            {row.row_id}')
        print(f'    vector:            {str(row.vector)[:64]} ...')
        print(f'    body_blob:         {row.body_blob[:64]} ...')
        print(f'    metadata_s:        {row.metadata_s}')        
    except AttributeError:
        # Please upgrade your cassIO to the latest version ...
        print(f'    document_id:      {row.document_id}')
        print(f'    embedding_vector: {str(row.embedding_vector)[:64]} ...')
        print(f'    document:         {row.document[:64]} ...')
        print(f'    metadata_blob:    {row.metadata_blob}')

print('\n...')


Row 0:
    row_id:            992f28c79fa541a6a0a58b4eafc86921
    vector:            [-0.0026160315610468388, -0.026444554328918457, 0.01814904808998 ...
    body_blob:         ---
title: "client.delete_webhook"
slug: "clientdelete_webhook"
 ...
    metadata_s:        None

Row 1:
    row_id:            d03e2f3fe2514a4c8a52229fb6b13f48
    vector:            [-0.02673957124352455, 0.006959426216781139, -0.0084350444376468 ...
    body_blob:         slug: "publishing-events-with-complex-data-formats" ,
           ...
    metadata_s:        None

Row 2:
    row_id:            3ea3314d6ebd4395ab423dab14f9b6a2
    vector:            [-0.015294374898076057, 0.006210180465131998, -0.007820494472980 ...
    body_blob:         ---
title: "Class-Imbalanced Data"
slug: "class-imbalanced-data" ...
    metadata_s:        None

Row 3:
    row_id:            08798909cdb94a6992590d78eda3ab0f
    vector:            [-0.031133130192756653, -0.007197968661785126, 0.010354483500123 ...
    body_blob:  

In [ ]:
query = "What is Fiddler?"
index.query(query, llm=llm)

In [ ]:
createTableSQL = """CREATE TABLE fiddler_chatbot_history (
    row_id text PRIMARY KEY,
    response text,
    response_vector vector<float, 1536>,
    source_docs text,
    source_docs_vector vector<float, 1536>,
    question text,
    question_vector vector<float, 1536>,
    comment text,
    feedback int,
    metadata_s map<text, text>,
    ts timestamp)"""
# ) WITH additional_write_policy = '99p'
#     AND bloom_filter_fp_chance = 0.01
#     AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
#     AND comment = ''
#     AND compaction = {'class': 'org.apache.cassandra.db.compaction.UnifiedCompactionStrategy'}
#     AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
#     AND crc_check_chance = 1.0
#     AND default_time_to_live = 0
#     AND gc_grace_seconds = 864000
#     AND max_index_interval = 2048
#     AND memtable_flush_period_in_ms = 0
#     AND min_index_interval = 128
#     AND read_repair = 'BLOCKING'
#     AND speculative_retry = '99p';"""

In [ ]:
session.execute(createTableSQL)

In [ ]:
EMBEDDING_MODEL = "text-embedding-ada-002"
response = openai.Embedding.create(model=EMBEDDING_MODEL, input='How are you doing')
response

In [ ]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

In [ ]:
rows = session.execute('SELECT * from squad')
    
df_baseline = rows._current_rows
df_baseline

In [ ]:
column_types = df_baseline.dtypes

print(column_types)

In [ ]:
df_baseline['answers'] = df_baseline['answers'].apply(lambda x : str(x))